In [2]:
!pip install sentencepiece

   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   ------------------------------- -------- 786.4/992.0 kB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 992.0/992.0 kB 7.8 MB/s eta 0:00:00


In [4]:
!pip install sentencepiece googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 13.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17516 sha256=505232c3fc959a91e3515cfa9b9e07c827c377c3b2fa2e8ea7eb8baca0adb00b
  Stored in directory: c:\users\ma698\appdata\local\pip\cache\wheels\95\0f\04\b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.5 requires httpx>=0.25.0, but you have httpx 0.13.3 which is incompatible.


In [5]:
!pip install transformers torch torchaudio jiwer

In [6]:
import os
import torch
import torchaudio
from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from jiwer import wer, cer
import warnings

# Ignore CUDA provider warnings since we're running on CPU
warnings.filterwarnings("ignore", message=".*CUDAExecutionProvider.*")

def load_translation_model():
    """Load Hindi to English translation model"""
    print("Loading translation model...")
    
    # Option 1: Using Helsinki-NLP model (recommended)
    model_name = "Helsinki-NLP/opus-mt-hi-en"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    translation_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    
    # Create translation pipeline
    translator = pipeline("translation", 
                         model=translation_model, 
                         tokenizer=tokenizer,
                         device=0 if torch.cuda.is_available() else -1)
    
    print("Translation model loaded successfully!")
    return translator

def translate_text(translator, hindi_text):
    """Translate Hindi text to English"""
    try:
        # Split long text into chunks if needed (model has token limits)
        max_length = 400  # Adjust based on model capacity
        
        if len(hindi_text) <= max_length:
            result = translator(hindi_text, max_length=512, num_beams=4, early_stopping=True)
            return result[0]['translation_text']
        else:
            # Split into sentences and translate each
            sentences = hindi_text.split('।')  # Hindi sentence delimiter
            if len(sentences) == 1:
                sentences = hindi_text.split('.')  # Fallback to period
            
            translated_sentences = []
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence:
                    result = translator(sentence, max_length=512, num_beams=4, early_stopping=True)
                    translated_sentences.append(result[0]['translation_text'])
            
            return ' '.join(translated_sentences)
    
    except Exception as e:
        print(f"Translation error: {e}")
        return None

def load_grounds_truth(file_path):
    """Load ground truth transcriptions from file"""
    if not os.path.exists(file_path):
        print(f"Ground truth file '{file_path}' not found!")
        return {}
    
    grounds_truths = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            
            # Handle different file formats
            if "|" in line:
                parts = line.split("|", 1)
                audio_file, transcription = parts[0].strip(), parts[1].strip()
                if not audio_file:
                    audio_file = "audio.flac"
            else:
                # No separator found, assume transcription for default audio file
                audio_file = "audio.flac"
                transcription = line
            
            grounds_truths[audio_file] = transcription
    
    return grounds_truths

def preprocess_audio(audio_path, target_sr=16000):
    """Load and preprocess audio file"""
    try:
        waveform, sr = torchaudio.load(audio_path)
        
        # Convert stereo to mono if needed
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        
        # Resample if needed
        if sr != target_sr:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
            waveform = resampler(waveform)
        
        return waveform, sr
    except Exception as e:
        print(f"Error loading audio: {e}")
        return None, None

def transcribe_audio(model, waveform):
    """Transcribe audio using the ASR model"""
    # Try different model calling approaches
    approaches = [
        lambda: model(waveform, "hi", "ctc"),
        lambda: model(waveform.squeeze(), "hi", "ctc"),
        lambda: model(waveform.unsqueeze(0), "hi", "ctc"),
    ]
    
    for approach in approaches:
        try:
            result = approach()
            return result if isinstance(result, str) else str(result)
        except:
            continue
    
    raise Exception("All transcription approaches failed")

def save_translations(translations, output_file="translations.txt"):
    """Save translations to file"""
    with open(output_file, 'w', encoding='utf-8') as f:
        for audio_file, data in translations.items():
            f.write(f"Audio File: {audio_file}\n")
            f.write(f"Hindi: {data['hindi']}\n")
            f.write(f"English: {data['english']}\n")
            f.write("-" * 80 + "\n")
    print(f"Translations saved to {output_file}")

def main():
    # Configuration
    gt_file = "grounds_truth.txt"
    target_sr = 16000
    output_file = "translations.txt"
    
    # Load models
    print("Loading ASR model...")
    asr_model = AutoModel.from_pretrained("ai4bharat/indic-conformer-600m-multilingual", trust_remote_code=True)
    print("ASR model loaded successfully!")
    
    translator = load_translation_model()
    
    # Load ground truth data (if available)
    grounds_truths = load_grounds_truth(gt_file)
    
    # Dictionary to store all translations
    all_translations = {}
    
    # If we have ground truth, process those files
    if grounds_truths:
        print(f"\nFound {len(grounds_truths)} audio file(s) to process\n")
        
        for audio_file, gt_text in grounds_truths.items():
            print(f"Processing: {audio_file}")
            
            # Find audio file
            if not os.path.exists(audio_file):
                print(f"Audio file not found: {audio_file}")
                # If we have ground truth text, translate it anyway
                print("Using ground truth text for translation...")
                hindi_text = gt_text
            else:
                # Load and transcribe audio
                waveform, original_sr = preprocess_audio(audio_file, target_sr)
                if waveform is None:
                    continue
                
                print(f"Audio loaded: {waveform.shape[1] / target_sr:.1f}s at {original_sr}Hz")
                
                try:
                    hindi_text = transcribe_audio(asr_model, waveform)
                    hindi_text = hindi_text.strip()
                    print(f"Transcription: {hindi_text}")
                except Exception as e:
                    print(f"Transcription failed: {e}")
                    continue
            
            # Translate to English
            print("Translating to English...")
            english_text = translate_text(translator, hindi_text)
            
            if english_text:
                print(f"Translation: {english_text}")
                all_translations[audio_file] = {
                    'hindi': hindi_text,
                    'english': english_text
                }
                
                # Calculate WER/CER if ground truth available
                if gt_text:
                    sample_wer = wer(gt_text, hindi_text)
                    sample_cer = cer(gt_text, hindi_text)
                    print(f"WER: {sample_wer:.3f} | CER: {sample_cer:.3f}")
            else:
                print("Translation failed!")
            
            print("-" * 80)
    
    else:
        # Process default audio file if no ground truth
        audio_file = "audio.flac"
        if os.path.exists(audio_file):
            print(f"Processing: {audio_file}")
            
            waveform, original_sr = preprocess_audio(audio_file, target_sr)
            if waveform is not None:
                print(f"Audio loaded: {waveform.shape[1] / target_sr:.1f}s at {original_sr}Hz")
                
                try:
                    hindi_text = transcribe_audio(asr_model, waveform)
                    hindi_text = hindi_text.strip()
                    print(f"Transcription: {hindi_text}")
                    
                    # Translate to English
                    print("Translating to English...")
                    english_text = translate_text(translator, hindi_text)
                    
                    if english_text:
                        print(f"Translation: {english_text}")
                        all_translations[audio_file] = {
                            'hindi': hindi_text,
                            'english': english_text
                        }
                    else:
                        print("Translation failed!")
                        
                except Exception as e:
                    print(f"Processing failed: {e}")
        else:
            print("No audio files found to process!")
    
    # Save all translations
    if all_translations:
        save_translations(all_translations, output_file)
        
        print(f"\nSUMMARY:")
        print(f"Files processed: {len(all_translations)}")
        print(f"Translations saved to: {output_file}")
        
        # Display final translations
        print("\nFINAL TRANSLATIONS:")
        for audio_file, data in all_translations.items():
            print(f"\nFile: {audio_file}")
            print(f"Hindi: {data['hindi']}")
            print(f"English: {data['english']}")
    else:
        print("No translations were generated!")

# Alternative function to translate existing Hindi text directly
def translate_hindi_text(hindi_text):
    """Standalone function to translate Hindi text to English"""
    translator = load_translation_model()
    english_text = translate_text(translator, hindi_text)
    return english_text

# Example usage for direct translation
def example_direct_translation():
    """Example of direct translation without audio processing"""
    # Your transcribed Hindi text
    hindi_text = "हेलो मेरा नाम मोहम्मद अतर है मुझे तीन दिन से खांसी और ज़ुकाम की शिकायत हो रही है मेरी बॉडी में भी पेन हो रहा है और गले में खराशे सी लग रही हैं साथ ही साथ सर भी भारी भारी रह रहा है तो क्या मुझे कोविड है या फिर कुछ और बीमारी होने का"
    
    print("Direct Translation Example:")
    print(f"Hindi: {hindi_text}")
    
    english_translation = translate_hindi_text(hindi_text)
    print(f"English: {english_translation}")

if __name__ == "__main__":
    # Run main program
    main()
    
    # Uncomment below to run direct translation example
    # print("\n" + "="*80)
    # example_direct_translation()

Loading ASR model...


Fetching 403 files:   0%|          | 0/403 [00:00<?, ?it/s]

ASR model loaded successfully!
Loading translation model...


ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

In [7]:
import os
import torch
import torchaudio
from transformers import AutoModel
from jiwer import wer, cer
import warnings

# Ignore CUDA provider warnings since we're running on CPU
warnings.filterwarnings("ignore", message=".*CUDAExecutionProvider.*")

def load_translation_model():
    """Load translation model with simple fallback options"""
    print("Loading translation model...")
    
    # Option 1: Try Google Translate (easiest, requires internet)
    try:
        from googletrans import Translator
        translator = Translator()
        # Test the translator
        test = translator.translate("हेलो", src='hi', dest='en')
        print("Google Translate loaded successfully!")
        return ("google", translator)
    except ImportError:
        print("Google Translate not available. Installing...")
        try:
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "googletrans==4.0.0-rc1"])
            from googletrans import Translator
            translator = Translator()
            print("Google Translate installed and loaded successfully!")
            return ("google", translator)
        except Exception as e:
            print(f"Failed to install Google Translate: {e}")
    except Exception as e:
        print(f"Google Translate error: {e}")
    
    # Option 2: Simple dictionary-based translation (basic fallback)
    print("Using basic dictionary translation (limited accuracy)")
    return ("basic", create_basic_translator())

def create_basic_translator():
    """Create a basic Hindi-English dictionary translator"""
    basic_dict = {
        'हेलो': 'hello',
        'मेरा': 'my',
        'नाम': 'name',
        'है': 'is',
        'मुझे': 'me/I',
        'दिन': 'day',
        'से': 'from',
        'खांसी': 'cough',
        'और': 'and',
        'ज़ुकाम': 'cold',
        'की': 'of',
        'शिकायत': 'complaint',
        'हो': 'is/being',
        'रही': 'happening',
        'मेरी': 'my',
        'बॉडी': 'body',
        'में': 'in',
        'भी': 'also',
        'पेन': 'pain',
        'रहा': 'happening',
        'गले': 'throat',
        'खराशे': 'irritation',
        'सी': 'like',
        'लग': 'feeling',
        'साथ': 'with',
        'ही': 'only/same',
        'सर': 'head',
        'भारी': 'heavy',
        'तो': 'so/then',
        'क्या': 'what',
        'कोविड': 'COVID',
        'या': 'or',
        'फिर': 'then',
        'कुछ': 'some',
        'बीमारी': 'disease',
        'होने': 'happening',
        'का': 'of'
    }
    return basic_dict

def translate_text(translator_info, hindi_text):
    """Translate Hindi text to English"""
    if translator_info is None:
        return "Translation not available"
    
    translator_type, translator = translator_info
    
    try:
        if translator_type == "google":
            # Google Translate
            result = translator.translate(hindi_text, src='hi', dest='en')
            return result.text
        
        elif translator_type == "basic":
            # Basic dictionary translation
            words = hindi_text.split()
            translated_words = []
            for word in words:
                # Remove punctuation for lookup
                clean_word = word.strip('।.,?!').lower()
                if clean_word in translator:
                    translated_words.append(translator[clean_word])
                else:
                    translated_words.append(f"[{word}]")  # Keep untranslated words in brackets
            
            return " ".join(translated_words)
        
        else:
            return "Unknown translation method"
    
    except Exception as e:
        print(f"Translation error: {e}")
        return f"Translation failed: {str(e)}"

def load_grounds_truth(file_path):
    """Load ground truth transcriptions from file"""
    if not os.path.exists(file_path):
        print(f"Ground truth file '{file_path}' not found!")
        return {}
    
    grounds_truths = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            
            # Handle different file formats
            if "|" in line:
                parts = line.split("|", 1)
                audio_file, transcription = parts[0].strip(), parts[1].strip()
                if not audio_file:
                    audio_file = "audio.flac"
            else:
                # No separator found, assume transcription for default audio file
                audio_file = "audio.flac"
                transcription = line
            
            grounds_truths[audio_file] = transcription
    
    return grounds_truths

def preprocess_audio(audio_path, target_sr=16000):
    """Load and preprocess audio file"""
    try:
        waveform, sr = torchaudio.load(audio_path)
        
        # Convert stereo to mono if needed
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        
        # Resample if needed
        if sr != target_sr:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
            waveform = resampler(waveform)
        
        return waveform, sr
    except Exception as e:
        print(f"Error loading audio: {e}")
        return None, None

def transcribe_audio(model, waveform):
    """Transcribe audio using the ASR model"""
    # Try different model calling approaches
    approaches = [
        lambda: model(waveform, "hi", "ctc"),
        lambda: model(waveform.squeeze(), "hi", "ctc"),
        lambda: model(waveform.unsqueeze(0), "hi", "ctc"),
    ]
    
    for approach in approaches:
        try:
            result = approach()
            return result if isinstance(result, str) else str(result)
        except:
            continue
    
    raise Exception("All transcription approaches failed")

def save_translations(translations, output_file="translations.txt"):
    """Save translations to file"""
    with open(output_file, 'w', encoding='utf-8') as f:
        for audio_file, data in translations.items():
            f.write(f"Audio File: {audio_file}\n")
            f.write(f"Hindi: {data['hindi']}\n")
            f.write(f"English: {data['english']}\n")
            f.write("-" * 80 + "\n")
    print(f"Translations saved to {output_file}")

def main():
    # Configuration
    gt_file = "grounds_truth.txt"
    target_sr = 16000
    output_file = "translations.txt"
    
    # Load ASR model
    print("Loading ASR model...")
    asr_model = AutoModel.from_pretrained("ai4bharat/indic-conformer-600m-multilingual", trust_remote_code=True)
    print("ASR model loaded successfully!")
    
    # Load translation model
    translator = load_translation_model()
    
    # Dictionary to store all translations
    all_translations = {}
    
    # Load ground truth data (if available)
    grounds_truths = load_grounds_truth(gt_file)
    
    # If we have ground truth, process those files
    if grounds_truths:
        print(f"\nFound {len(grounds_truths)} audio file(s) to process\n")
        
        for audio_file, gt_text in grounds_truths.items():
            print(f"Processing: {audio_file}")
            
            # Find audio file
            if not os.path.exists(audio_file):
                print(f"Audio file not found: {audio_file}")
                # If we have ground truth text, translate it anyway
                print("Using ground truth text for translation...")
                hindi_text = gt_text
            else:
                # Load and transcribe audio
                waveform, original_sr = preprocess_audio(audio_file, target_sr)
                if waveform is None:
                    continue
                
                print(f"Audio loaded: {waveform.shape[1] / target_sr:.1f}s at {original_sr}Hz")
                
                try:
                    hindi_text = transcribe_audio(asr_model, waveform)
                    hindi_text = hindi_text.strip()
                    print(f"Transcription: {hindi_text}")
                except Exception as e:
                    print(f"Transcription failed: {e}")
                    continue
            
            # Translate to English
            print("Translating to English...")
            english_text = translate_text(translator, hindi_text)
            
            print(f"Translation: {english_text}")
            all_translations[audio_file] = {
                'hindi': hindi_text,
                'english': english_text
            }
            
            # Calculate WER/CER if ground truth available
            if gt_text and hindi_text != gt_text:
                sample_wer = wer(gt_text, hindi_text)
                sample_cer = cer(gt_text, hindi_text)
                print(f"WER: {sample_wer:.3f} | CER: {sample_cer:.3f}")
            
            print("-" * 80)
    
    else:
        # Process default audio file if no ground truth
        audio_file = "audio.flac"
        if os.path.exists(audio_file):
            print(f"Processing: {audio_file}")
            
            waveform, original_sr = preprocess_audio(audio_file, target_sr)
            if waveform is not None:
                print(f"Audio loaded: {waveform.shape[1] / target_sr:.1f}s at {original_sr}Hz")
                
                try:
                    hindi_text = transcribe_audio(asr_model, waveform)
                    hindi_text = hindi_text.strip()
                    print(f"Transcription: {hindi_text}")
                    
                    # Translate to English
                    print("Translating to English...")
                    english_text = translate_text(translator, hindi_text)
                    
                    print(f"Translation: {english_text}")
                    all_translations[audio_file] = {
                        'hindi': hindi_text,
                        'english': english_text
                    }
                        
                except Exception as e:
                    print(f"Processing failed: {e}")
            
            all_translations["sample"] = {
                'hindi': sample_text,
                'english': english_text
            }
    
    # Save all translations
    if all_translations:
        save_translations(all_translations, output_file)
        
        print(f"\nSUMMARY:")
        print(f"Files processed: {len(all_translations)}")
        print(f"Translations saved to: {output_file}")
        
        # Display final translations
        print("\nFINAL TRANSLATIONS:")
        for audio_file, data in all_translations.items():
            print(f"\nFile: {audio_file}")
            print(f"Hindi: {data['hindi']}")
            print(f"English: {data['english']}")
    else:
        print("No translations were generated!")

# Simple function to translate just text
def quick_translate(hindi_text):
    """Quick translation function"""
    translator = load_translation_model()
    return translate_text(translator, hindi_text)

if __name__ == "__main__":
    main()

Loading ASR model...


Fetching 403 files:   0%|          | 0/403 [00:00<?, ?it/s]

ASR model loaded successfully!
Loading translation model...
Google Translate loaded successfully!

Found 1 audio file(s) to process

Processing: audio.flac
Audio loaded: 25.7s at 44100Hz
Transcription: हेलो मेरा नाम मोहम्मद अतर है मुझे तीन दिन से खांसी और ज़ुकाम की शिकायत हो रही है मेरी बॉडी में भी पेन हो रहा है और गले में हराशें स ही लग रही हैं साथ ही साथ सर भी भारी भारी रह रहा है तो क्या मुझे कोविड है या फिर कुछ और बीमारी होने का
Translating to English...
Translation: Hello, my name is Mohammad Atar, I have been complaining of cough and cold for three days, my body is also getting pen and there is only a green in the neck, as well as the head is also heavy, so do I have covid or some other disease
WER: 0.089 | CER: 0.026
--------------------------------------------------------------------------------
Translations saved to translations.txt

SUMMARY:
Files processed: 1
Translations saved to: translations.txt

FINAL TRANSLATIONS:

File: audio.flac
Hindi: हेलो मेरा नाम मोहम्मद अतर है मुझ

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def translate_hindi_to_english(hindi_text):
    """Translate Hindi text to English using Helsinki-NLP model"""
    
    # Load the translation model
    model_name = "Helsinki-NLP/opus-mt-hi-en"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    
    # Tokenize and translate
    inputs = tokenizer(hindi_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    
    # Decode the translation
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return translation

# Input: Predicted transcription from your ASR model
hindi_transcription = "हेलो मेरा नाम मोहम्मद अतर है मुझे तीन दिन से खांसी और ज़ुकाम की शिकायत हो रही है मेरी बॉडी में भी पेन हो रहा है और गले में हराशें स ही लग रही हैं साथ ही साथ सर भी भारी भारी रह रहा है तो क्या मुझे कोविड है या फिर कुछ और बीमारी होने का"

# Translate to English
english_translation = translate_hindi_to_english(hindi_transcription)

print("Original Hindi:")
print(hindi_transcription)
print("\nEnglish Translation:")
print(english_translation)

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

In [9]:
import json
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def extract_prediction_from_notebook(notebook_file):
    """Extract predicted transcription from Jupyter notebook file"""
    try:
        with open(notebook_file, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
        
        # Look through all cells for prediction output
        for cell in notebook['cells']:
            if 'outputs' in cell:
                for output in cell['outputs']:
                    if 'text' in output:
                        text_lines = output['text']
                        for line in text_lines:
                            if line.startswith('Prediction:'):
                                # Extract text after "Prediction: "
                                prediction = line.replace('Prediction:', '').strip()
                                return prediction
        
        print("No prediction found in notebook file")
        return None
        
    except Exception as e:
        print(f"Error reading notebook file: {e}")
        return None

def translate_hindi_to_english(hindi_text):
    """Translate Hindi text to English using Helsinki-NLP model"""
    
    # Load the translation model
    model_name = "Helsinki-NLP/opus-mt-hi-en"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    
    # Tokenize and translate
    inputs = tokenizer(hindi_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    
    # Decode the translation
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return translation

# Automatically extract prediction from notebook file
notebook_file = "med_audi_move.ipynb"
hindi_transcription = extract_prediction_from_notebook(notebook_file)

if hindi_transcription:
    print("Extracted Hindi Transcription:")
    print(hindi_transcription)
    
    # Translate to English
    print("\nTranslating...")
    english_translation = translate_hindi_to_english(hindi_transcription)
    
    print("\nEnglish Translation:")
    print(english_translation)
else:
    print("Could not extract transcription from notebook file")

Extracted Hindi Transcription:
हेलो मेरा नाम मोहम्मद अतर है मुझे तीन दिन से खांसी और ज़ुकाम की शिकायत हो रही है मेरी बॉडी में भी पेन हो रहा है और गले में हराशें स ही लग रही हैं साथ ही साथ सर भी भारी भारी रह रहा है तो क्या मुझे कोविड है या फिर कुछ और बीमारी होने का

Translating...


ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

In [10]:
import json
import re
import json
from googletrans import Translator

def extract_prediction_from_notebook(notebook_file):
    """Extract predicted transcription from Jupyter notebook file"""
    try:
        with open(notebook_file, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
        
        # Look through all cells for prediction output
        for cell in notebook['cells']:
            if 'outputs' in cell:
                for output in cell['outputs']:
                    if 'text' in output:
                        text_lines = output['text']
                        for line in text_lines:
                            if line.startswith('Prediction:'):
                                # Extract text after "Prediction: "
                                prediction = line.replace('Prediction:', '').strip()
                                return prediction
        
        print("No prediction found in notebook file")
        return None
        
    except Exception as e:
        print(f"Error reading notebook file: {e}")
        return None

def translate_hindi_to_english(hindi_text):
    """Translate Hindi text to English using Google Translate"""
    try:
        translator = Translator()
        result = translator.translate(hindi_text, src='hi', dest='en')
        return result.text
    except Exception as e:
        print(f"Translation error: {e}")
        return None

# Automatically extract prediction from notebook file
notebook_file = "med_audi_move.ipynb"
hindi_transcription = extract_prediction_from_notebook(notebook_file)

if hindi_transcription:
    print("Extracted Hindi Transcription:")
    print(hindi_transcription)
    
    # Translate to English
    print("\nTranslating...")
    english_translation = translate_hindi_to_english(hindi_transcription)
    
    print("\nEnglish Translation:")
    print(english_translation)
else:
    print("Could not extract transcription from notebook file")

Extracted Hindi Transcription:
हेलो मेरा नाम मोहम्मद अतर है मुझे तीन दिन से खांसी और ज़ुकाम की शिकायत हो रही है मेरी बॉडी में भी पेन हो रहा है और गले में हराशें स ही लग रही हैं साथ ही साथ सर भी भारी भारी रह रहा है तो क्या मुझे कोविड है या फिर कुछ और बीमारी होने का

Translating...

English Translation:
Hello, my name is Mohammad Atar, I have been complaining of cough and cold for three days, my body is also getting pen and there is only a green in the neck, as well as the head is also heavy, so do I have covid or some other disease
